# [Deutsch-Jozsa Algorithm](../notebooks/deutsch_jozsa.ipynb) #

The methods for developing the circuits can be found in [src/algorithms/deutsch_jozsa.py](../src/algorithms/deutsch_jozsa.py).

## Demonstration ##

See [examples/deutsch_jozsa.md](../examples/deutsch_jozsa.md)

In [ ]:
'''IMPORTS'''
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.quantum import *;
from src.setup import *;
from src.demo.deutsch_jozsa import *;

set_rng_seed(); # for repeatability

In [ ]:
'''Example with simulator'''
action_prepare_circuit_and_job(
    option = BACKEND_SIMULATOR.AER,
    num_shots = 1000,
    n = 4,
);

action_display_statistics(queue=False, as_widget=False);

In [ ]:
'''Example with IBM cloud backend (queue)'''
action_prepare_circuit_and_job(
    option = BACKEND.LEAST_BUSY,
    num_shots = 10000,
    n = 4,
);

action_display_statistics(queue=True, as_widget=True);

In [ ]:
'''Statistics for backend job - NOTE: job may be pending'''
action_display_statistics(
    queue = True,
    as_widget = False,
    # Use following to manually select a backend + job; otherwise tries to retrieve last job.
    # job_id = 'abcd1234', 
    # backend_option = BACKEND. ,
);